In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 512
channel = 96
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [5]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
eeg_dataset, splits_path, splits_shuffled_path = os.listdir(torch_models_dir)

In [6]:
eeg_dataset = os.path.join(torch_models_dir, eeg_dataset)
splits_path = os.path.join(torch_models_dir, splits_path)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_path)

/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth 
 /media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth


In [7]:
opt = {
    # Dataset options
    "iv": "image",
    "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "run": "none",
    "eeg_dataset": eeg_dataset,
    "splits_path": splits_path,
    "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"incremental",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.iv)

image


In [18]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from data_loader import EEGDataset, Splitter, SplitterWithData
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.LSTM_per_channel import classifier_LSTM_per_channel
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def load_dataset(iv,
             offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-4) - 5 fold cross validation
             total,
             classes,
             classifier,
             batch_size,
             GPUindex,
             length, # 512
             channel, # 96
             min_CNN,
             opt,
             kind):        
    # Load dataset
    dataset = EEGDataset(iv, eeg_dataset, classifier, map_idx = None)
    print("DONE: LOAD DATASET")
    # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
    if kind=="from-scratch":
        relabel = False
    if kind=="incremental":
        relabel = False
    if kind=="no-model-file":
        relabel = True
    splitter = {split: SplitterWithData(iv,
                    dataset,
                    splits_path,
                    classes,
                    split_num,
                    split,
                    relabel) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [10]:
# LSTM per channel network
opt.classifier = "LSTM_per_channel"
opt.batch_size = 16
opt.kind = "from-scratch"
opt.run = "imagenet40-1000"
opt.fold = 5

In [11]:
dataset, loaders, splitter = load_dataset(opt.iv,
                             opt.offset,
                             opt.eeg_dataset,
                             opt.splits_path,
                             0, #split_num
                             n_classes,
                             classes,
                             opt.classifier,
                             opt.batch_size,
                             opt.GPUindex,
                             length,
                             channel,
                             min_CNN,
                             opt,
                             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [13]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 32000 idx, val: 4000 idx, test: 4000 idx}
# Each loader batch has {train: 2000 idx, val: 250idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i in range(0, 40):
    eeg, label_val = splitter["val"][i*100]
    eeg, label_train = splitter["train"][i*800]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")
# print(splitter["val"].split_idx[:10])

<class 'data_loader.EEGDataset'>
<class 'dict'>
3 [2000, 250, 250]
1: Label val: 0; label train: 0
2: Label val: 17; label train: 17
3: Label val: 28; label train: 28
4: Label val: 7; label train: 7
5: Label val: 33; label train: 33
6: Label val: 12; label train: 12
7: Label val: 21; label train: 21
8: Label val: 3; label train: 3
9: Label val: 25; label train: 25
10: Label val: 36; label train: 36
11: Label val: 10; label train: 10
12: Label val: 15; label train: 15
13: Label val: 19; label train: 19
14: Label val: 31; label train: 31
15: Label val: 23; label train: 23
16: Label val: 5; label train: 5
17: Label val: 38; label train: 38
18: Label val: 8; label train: 8
19: Label val: 1; label train: 1
20: Label val: 34; label train: 34
21: Label val: 29; label train: 29
22: Label val: 26; label train: 26
23: Label val: 13; label train: 13
24: Label val: 11; label train: 11
25: Label val: 22; label train: 22
26: Label val: 18; label train: 18
27: Label val: 6; label train: 6
28: Label v

In [14]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1, 96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_LSTM_per_channel(
  (lstm_per_channel): LSTM(1, 1, batch_first=True)
  (lstm_full1): LSTM(96, 128, batch_first=True)
  (lstm_full2): LSTM(128, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=40, bias=True)
  (softmax): Softmax(dim=None)
)
torch.Size([1, 96, 512])


/home/titan/GithubClonedRepo/EEG-Research/Experiment/EEG_Encoder/LSTM_per_channel.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_in = torch.tensor(X[:, i, :]) # x_in: (batch_size, 1, timesteps)


X after concat:  torch.Size([1, 512, 96])


/home/titan/GithubClonedRepo/EEG-Research/Experiment/EEG_Encoder/LSTM_per_channel.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = self.softmax(X)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_LSTM_per_channel              [1, 40]                   --
├─LSTM: 1-1                              [1, 512, 1]               16
├─LSTM: 1-2                              [1, 512, 1]               (recursive)
├─LSTM: 1-3                              [1, 512, 1]               (recursive)
├─LSTM: 1-4                              [1, 512, 1]               (recursive)
├─LSTM: 1-5                              [1, 512, 1]               (recursive)
├─LSTM: 1-6                              [1, 512, 1]               (recursive)
├─LSTM: 1-7                              [1, 512, 1]               (recursive)
├─LSTM: 1-8                              [1, 512, 1]               (recursive)
├─LSTM: 1-9                              [1, 512, 1]               (recursive)
├─LSTM: 1-10                             [1, 512, 1]               (recursive)
├─LSTM: 1-11                             [1, 512, 1]               (recurs

In [15]:
model_path = (opt.iv+
                  "-"+
                  opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel)+
                  "-"+
                  str(0)) + "-noshuffle"
                  
channel_idx=None
print(model_path)

image-LSTM_per_channel-512-96-0-noshuffle


In [16]:
print(opt)

Namespace(iv='image', offset=None, results_file='results.pkl', subject=0, run='imagenet40-1000', eeg_dataset='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1.pth', splits_path='/media/titan/AI Research/Data/CVPR2021-02785/CVPR2021-02785/preprocessed/torch_models/imagenet40-1000-1_splits.pth', fold=5, batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='LSTM_per_channel')


In [19]:
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

epoch 1
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])


/home/titan/GithubClonedRepo/EEG-Research/Experiment/EEG_Encoder/LSTM_per_channel.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_in = torch.tensor(X[:, i, :]) # x_in: (batch_size, 1, timesteps)
/home/titan/GithubClonedRepo/EEG-Research/Experiment/EEG_Encoder/LSTM_per_channel.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = self.softmax(X)


Batch 1000: Loss=3.6873857975006104; accuracy=0.02537500113248825
Batch 2000: Loss=3.6891353130340576; accuracy=0.024625001475214958
Validation accuracy: 0.02500000037252903
epoch 2
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.687983751296997; accuracy=0.0240625012665987
Batch 2000: Loss=3.6889331340789795; accuracy=0.02331250160932541
Validation accuracy: 0.02500000037252903
epoch 3
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.689671277999878; accuracy=0.024000000208616257
Batch 2000: Loss=3.689314126968384; accuracy=0.023000001907348633
Validation accuracy: 0.02500000037252903
epoch 4
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6888632774353027; accuracy=0.02550000138580799
Batch 2000: Loss=3.6890547275543213; accuracy=0.02

Batch 1000: Loss=3.6834816932678223; accuracy=0.027375001460313797
Batch 2000: Loss=3.692352533340454; accuracy=0.027937501668930054
Validation accuracy: 0.023250000551342964
epoch 30
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6792259216308594; accuracy=0.0266875009983778
Batch 2000: Loss=3.686769485473633; accuracy=0.026718750596046448
Validation accuracy: 0.026250001043081284
epoch 31
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6857948303222656; accuracy=0.029000001028180122
Batch 2000: Loss=3.6874165534973145; accuracy=0.028031250461935997
Validation accuracy: 0.027750002220273018
epoch 32
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.68756365776062; accuracy=0.026250001043081284
Batch 2000: Loss=3.681086540222168; accura

Batch 1000: Loss=3.69341778755188; accuracy=0.07443750649690628
Batch 2000: Loss=3.644908905029297; accuracy=0.07518750429153442
Validation accuracy: 0.024250000715255737
epoch 58
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.670081615447998; accuracy=0.08162500709295273
Batch 2000: Loss=3.651620388031006; accuracy=0.078125
Validation accuracy: 0.020750001072883606
epoch 59
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6886260509490967; accuracy=0.08275000751018524
Batch 2000: Loss=3.6537511348724365; accuracy=0.08125000447034836
Validation accuracy: 0.02225000038743019
epoch 60
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6445529460906982; accuracy=0.08362500369548798
Batch 2000: Loss=3.6698694229125977; accuracy=0.082812502980

Batch 1000: Loss=3.5722265243530273; accuracy=0.1420625001192093
Batch 2000: Loss=3.536834478378296; accuracy=0.14034375548362732
Validation accuracy: 0.02500000037252903
epoch 86
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6639950275421143; accuracy=0.15037500858306885
Batch 2000: Loss=3.582749128341675; accuracy=0.14665625989437103
Validation accuracy: 0.02500000037252903
epoch 87
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.565037965774536; accuracy=0.14787501096725464
Batch 2000: Loss=3.5544605255126953; accuracy=0.1472500115633011
Validation accuracy: 0.024250000715255737
epoch 88
Size of input:  torch.Size([16, 96, 512])
Size of target:  torch.Size([16])
Size of output:  torch.Size([16, 40])
Batch 1000: Loss=3.6433093547821045; accuracy=0.15031251311302185
Batch 2000: Loss=3.669513702392578; accuracy=0.149

In [20]:
val =accuracy_val
test = accuracy_test

print("Validation accuracy: ", val)
print("Test accuracy: ", test)

Validation accuracy:  0.024000000208616257
Test accuracy:  0


In [ ]:
print(accuracy_val)